In [1]:
import sys
sys.path.append("../../src")
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn.functional as F

import glob
import os
from datetime import datetime
import time
import math
from tqdm import tqdm

from itertools import repeat
from torch.nn.parameter import Parameter
import collections
import matplotlib
from torch_utils import *
from ContrastiveModels import ContrastiveCorInfoMaxHopfield
from visualization import *
# matplotlib.use('Agg')

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), 
                                            torchvision.transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), 
                                            std=(3*0.2023, 3*0.1994, 3*0.2010))])

cifar_dset_train = torchvision.datasets.CIFAR10('../../data', train=True, transform=transform, target_transform=None, download=True)
train_loader = torch.utils.data.DataLoader(cifar_dset_train, batch_size=20, shuffle=True, num_workers=0)

cifar_dset_test = torchvision.datasets.CIFAR10('../../data', train=False, transform=transform, target_transform=None, download=True)
test_loader = torch.utils.data.DataLoader(cifar_dset_test, batch_size=20, shuffle=False, num_workers=0)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
activation = hard_sigmoid
# architecture = [int(32*32*3), 500, 10]
architecture = [int(32*32*3), 1000, 500, 10]

x,y = next(iter(train_loader))
x = x.view(x.size(0),-1).to(device).T
y_one_hot = F.one_hot(y, 10).to(device).T

beta = 1
lambda_ = 0.99999
epsilon = 0.15
one_over_epsilon = 1 / epsilon
lr_start = {'ff': np.array([0.1, 0.05, 0.03]), 'fb': np.array([ np.nan, 0.045, 0.02])}

neural_lr_start = 0.05
neural_lr_stop = 0.001
neural_lr_rule = "constant"
neural_lr_decay_multiplier = 0.01
neural_dynamic_iterations_nudged = 10
neural_dynamic_iterations_free = 30
hopfield_g = 0.1
use_random_sign_beta = True
use_three_phase = False
weight_decay = False

model = ContrastiveCorInfoMaxHopfield(architecture = architecture, lambda_ = lambda_, 
                                      epsilon = epsilon, activation = activation)

In [5]:
_ = evaluateContrastiveCorInfoMaxHopfield(model, train_loader, hopfield_g,
                                          neural_lr_start, neural_lr_stop, neural_lr_rule, 
                                          neural_lr_decay_multiplier, neural_dynamic_iterations_free, device)

Train accuracy :	 0.0958


In [ ]:
trn_acc_list = []
tst_acc_list = []

n_epochs = 50

for epoch_ in range(n_epochs):
    if epoch_ < 15:
        lr = {'ff' : lr_start['ff'] * (0.95)**epoch_, 'fb' : lr_start['fb'] * (0.95)**epoch_}
    else:
        lr = {'ff' : lr_start['ff'] * (0.9)**epoch_, 'fb' : lr_start['fb'] * (0.9)**epoch_}
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x, y = x.to(device), y.to(device)
        x = x.view(x.size(0),-1).T
        y_one_hot = F.one_hot(y, 10).to(device).T
        take_debug_logs_ = (idx % 500 == 0)
        if use_random_sign_beta:
            rnd_sgn = 2*np.random.randint(2) - 1
            beta = rnd_sgn*beta
            
        neurons = model.batch_step_hopfield( x, y_one_hot, hopfield_g, 
                                             lr, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                                             neural_lr_decay_multiplier, neural_dynamic_iterations_free,
                                             neural_dynamic_iterations_nudged, beta, 
                                             use_three_phase, take_debug_logs_, weight_decay)
    
    trn_acc = evaluateContrastiveCorInfoMaxHopfield(model, train_loader, hopfield_g, neural_lr_start, 
                                                    neural_lr_stop, neural_lr_rule, 
                                                    neural_lr_decay_multiplier, 
                                                    neural_dynamic_iterations_free, 
                                                    device, printing = False)
    tst_acc = evaluateContrastiveCorInfoMaxHopfield(model, test_loader, hopfield_g, neural_lr_start, 
                                                    neural_lr_stop, neural_lr_rule, 
                                                    neural_lr_decay_multiplier, 
                                                    neural_dynamic_iterations_free, 
                                                    device, printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))
    print("Free Information ratio: {}".format(np.array(model.layerwise_forward_corinfo_list_free)[-1] / np.array(model.layerwise_backward_corinfo_list_free)[-1]))
    print("Nudged Information ratio: {}".format(np.array(model.layerwise_forward_corinfo_list_nudged)[-1] / np.array(model.layerwise_backward_corinfo_list_nudged)[-1]))

2500it [02:34, 16.19it/s]
0it [00:00, ?it/s]

Epoch : 1, Train Accuracy : 0.2825, Test Accuracy : 0.2894
Free Information ratio: [0.42460673 0.00069787]
Nudged Information ratio: [0.42460657 0.00069749]


2500it [02:34, 16.15it/s]
0it [00:00, ?it/s]

Epoch : 2, Train Accuracy : 0.35044, Test Accuracy : 0.3505
Free Information ratio: [0.47083491 0.00224455]
Nudged Information ratio: [0.4708345 0.0022437]


2500it [02:35, 16.10it/s]
0it [00:00, ?it/s]

...

2500it [02:34, 16.20it/s]
0it [00:00, ?it/s]

Epoch : 29, Train Accuracy : 0.56326, Test Accuracy : 0.4992
Free Information ratio: [0.6180577 0.0072692]
Nudged Information ratio: [0.61805302 0.00726065]


2500it [02:33, 16.24it/s]
0it [00:00, ?it/s]

Epoch : 30, Train Accuracy : 0.56342, Test Accuracy : 0.4989
Free Information ratio: [0.55736064 0.00742212]
Nudged Information ratio: [0.55735546 0.00741279]


2500it [02:33, 16.27it/s]


In [ ]:
def columnwise_sparsity(x, threshold = 0.01):
    return (x < threshold).sum(0) / x.shape[0]

In [ ]:
x, y = next(iter(train_loader))
x, y = x.to(device), y.to(device)

x = x.view(x.size(0),-1).T
y_one_hot = F.one_hot(y, 10).to(device).T
neurons = model.init_neurons(x.size(1), device = model.device)

neurons = model.run_neural_dynamics_hopfield(x, 0, neurons, hopfield_g, neural_lr_start, 
                                   neural_lr_stop, neural_lr_rule, 
                                   neural_lr_decay_multiplier, neural_dynamic_iterations_free, 
                                   0, False)

In [ ]:
columnwise_sparsity(neurons[0][0]).mean()

In [ ]:
# for epoch_ in range(n_epochs, n_epochs + 25):
#     lr = {'ff' : lr_start['ff'] * (0.95)**epoch_, 'fb' : lr_start['fb'] * (0.95)**epoch_}
#     for idx, (x, y) in tqdm(enumerate(train_loader)):
#         x, y = x.to(device), y.to(device)
#         x = x.view(x.size(0),-1).T
#         y_one_hot = F.one_hot(y, 10).to(device).T
#         take_debug_logs_ = (idx % 500 == 0)
#         if use_random_sign_beta:
#             rnd_sgn = 2*np.random.randint(2) - 1
#             beta = rnd_sgn*beta
            
#         neurons = model.batch_step_hopfield( x, y_one_hot, hopfield_g, 
#                                              lr, neural_lr_start, neural_lr_stop, neural_lr_rule, 
#                                              neural_lr_decay_multiplier, neural_dynamic_iterations_free,
#                                              neural_dynamic_iterations_nudged, beta, 
#                                              use_three_phase, take_debug_logs_, weight_decay)
    
#     trn_acc = evaluateContrastiveCorInfoMaxHopfield(model, train_loader, hopfield_g, neural_lr_start, 
#                                                     neural_lr_stop, neural_lr_rule, 
#                                                     neural_lr_decay_multiplier, 
#                                                     neural_dynamic_iterations_free, 
#                                                     device, printing = False)
#     tst_acc = evaluateContrastiveCorInfoMaxHopfield(model, test_loader, hopfield_g, neural_lr_start, 
#                                                     neural_lr_stop, neural_lr_rule, 
#                                                     neural_lr_decay_multiplier, 
#                                                     neural_dynamic_iterations_free, 
#                                                     device, printing = False)
#     trn_acc_list.append(trn_acc)
#     tst_acc_list.append(tst_acc)
    
#     print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))
#     print("Free Information ratio: {}".format(np.array(model.layerwise_forward_corinfo_list_free)[-1] / np.array(model.layerwise_backward_corinfo_list_free)[-1]))
#     print("Nudged Information ratio: {}".format(np.array(model.layerwise_forward_corinfo_list_nudged)[-1] / np.array(model.layerwise_backward_corinfo_list_nudged)[-1]))

In [ ]:
plot_convergence_plot(trn_acc_list, xlabel = 'Number of Epochs', ylabel = 'Accuracy %',
                      title = 'Contrastive CorInfoMax Train Accuracy w.r.t. Epochs', 
                      figsize = (12,8), fontsize = 25, linewidth = 3)

In [ ]:
plot_convergence_plot(tst_acc_list, xlabel = 'Number of Epochs', ylabel = 'Accuracy %',
                      title = 'Contrastive CorInfoMax Test Accuracy w.r.t. Epochs', 
                      figsize = (12,8), fontsize = 25, linewidth = 3)

In [ ]:
# from IPython.display import Math, display
# ########### LATEX Style Display Matrix ###############
# def display_matrix(array):
#     """Display given numpy array with Latex format in Jupyter Notebook.
#     Args:
#         array (numpy array): Array to be displayed
#     """
#     data = ""
#     for line in array:
#         if len(line) == 1:
#             data += " %.3f &" % line + r" \\\n"
#             continue
#         for element in line:
#             data += " %.3f &" % element
#         data += r" \\" + "\n"
#     display(Math("\\begin{bmatrix} \n%s\\end{bmatrix}" % data))

In [ ]:
# display_matrix(model.B[0]['weight'][:10,:10])

In [ ]:
# display_matrix(torch.linalg.inv(model.Rh1)[:10,:10])

In [ ]:
# torch.norm(model.B[0]['weight'] - torch.linalg.inv(model.Rh1))

In [ ]:
# display_matrix(torch.linalg.inv(model.Rh2)[:10,:10])